In [ ]:
##GENERAL
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import csv
import re
import os

##SPACY
import spacy
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_md')
#nlp = spacy.load('en_core_web_lg')
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.pipeline import SentenceSegmenter
from spacy import displacy

import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, Input,BatchNormalization, Dropout
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

##SKLEARN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.decomposition import NMF

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


##Loading in the data

In [ ]:
data = pd.read_csv('/gdrive/My Drive/data/amazon_products.csv')
data = data.drop(columns=['Unnamed: 0'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58828 entries, 0 to 58827
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   review           58828 non-null  object
 1   clean_review     58828 non-null  object
 2   review_category  58828 non-null  object
 3   topic_category   58828 non-null  int64 
 4   topic_name       58828 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.2+ MB


In [ ]:
data['review_category'].value_counts()

Bad        33406
Good       16535
Average     8887
Name: review_category, dtype: int64

In [ ]:
data['topic_name'].value_counts()

refund-and-return    23020
books                16273
music                 7768
video-quality         5203
movies                4855
games                 1709
Name: topic_name, dtype: int64

##Creating the Text to Categorical Columns

In [ ]:
data['topic_name']=data['topic_name'].astype('category')
data['topic_name'].cat.categories
data['topic_category_codes']=data['topic_name'].cat.codes.values
data = data.drop(columns=['review','review_category','topic_category','topic_name'])
data.head()

,clean_review,topic_category_codes
0,this self published book want know read paragr...,0
1,rather scratches insect droppings random pixel...,5
2,my experience got fm Amazon the dvd player rat...,5
3,ordered dvd received substitute received dvd o...,5
4,disappointed performance it underpowered const...,4


In [ ]:
#Transforamation from pandas colums to numpy array
sentences = data["clean_review"].fillna("DUMMY_VALUE").values
targets = data['topic_category_codes'].values
print(type(sentences))
print(type(targets))
print((sentences[0]))
print((targets[0]))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
this self published book want know read paragraphs those star reviews written ms haddon family friends imagine reading thing spent evening book friend hysterics reading bits pieces it definitely bad entered kind worst book contest believe Amazon sells kind thing maybe offer th grade term paper to kill mockingbird book sure ms haddon heard anyway mood send book joke stay far far away
0


##Declaring Variables

In [ ]:
# some configuration
MAX_SEQUENCE_LENGTH = 100
MAX_VOCAB_SIZE = 30000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 128
EPOCHS = 80
output_dir = '/gdrive/My Drive/amazon_review_full_csv'

##Creating Train and Test Dataset

In [ ]:
df_train, df_test, Ytrain, Ytest = train_test_split(sentences, targets, test_size=0.33)

##Tokenization and Padding

In [ ]:
# convert the sentences (strings) into integers
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, filters='')
tokenizer.fit_on_texts(df_train)
sequences_train = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)


In [ ]:
idx2word = tokenizer.index_word
#word2idx = {k:v for v,k in idx2word.items()}
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))
print(word2idx['instead'])
print(idx2word[157])

Found 64186 unique tokens.
141
happy


In [ ]:
seq_len = [len(sent) for sent in sequences_train]
max_seq_len = max(seq_len)
print(f'maximum sequencse length {max_seq_len}')
padded_data_train = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', padded_data_train.shape)
padded_data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data test tensor:', padded_data_test.shape)

maximum sequencse length 157
Shape of data tensor: (39414, 100)
Shape of data test tensor: (19414, 100)


##Creating the Embedding Matrix using Glove() pretrained vectors

In [ ]:
# load in pre-trained word vectors
print('Loading word vectors...')
word2vec = {}
with open(os.path.join('/gdrive/My Drive/data/glove.6B.%sd.txt' % EMBEDDING_DIM)) as f:
  for line in f:
    values = line.split()
    word = values[0]
    value = np.asarray(values[1:] , dtype='float32')
    word2vec[word]=value
print(len(word2vec))

Loading word vectors...
400000


In [ ]:
# prepare embedding matrix
#print('Filling pre-trained embeddings...')
num_words = min(MAX_VOCAB_SIZE, len(word2idx) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word,i in word2idx.items():
  #print(word)
  #print(i)
  if i < MAX_VOCAB_SIZE:
    embedding_vector= word2vec.get(word)
    if embedding_vector is not None:
      embedding_matrix[i]=embedding_vector
print(embedding_matrix.shape)

(30000, 50)


In [ ]:
embedding_layer = Embedding(
    num_words,
    EMBEDDING_DIM,
    weights=[embedding_matrix],
    input_length=MAX_SEQUENCE_LENGTH,
    trainable=False
)

In [ ]:
type(df_train)

numpy.ndarray

##Model Architecture

In [ ]:
input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
embeddings = embedding_layer(input_)
bilstm = Bidirectional(LSTM(32, return_sequences=True, return_state=False, dropout=0.2))(embeddings)
lstm = LSTM(64, return_sequences=True, return_state=False, dropout=0.2)
lstm_layer = lstm(bilstm)
gmpl= GlobalMaxPool1D(name='gmpl')(lstm_layer)
dense = Dense(3)
output = dense(gmpl)
model = Model(input_, output)

input_ = Input(shape=(MAX_SEQUENCE_LENGTH,))
embeddings = embedding_layer(input_)
lstm_1 = Bidirectional(LSTM(128, return_sequences=True, return_state=False))(embeddings)
dropout = Dropout(0.3)(lstm_1)
lstm_2 = Bidirectional(LSTM(256, return_sequences=True, return_state=False, dropout=0.3))
lstm_layer = lstm_2(dropout)
gmpl= GlobalMaxPool1D(name='gmpl')(lstm_layer)
dense = Dense(64,kernel_initializer=tf.keras.initializers.glorot_normal(seed=None),
    kernel_regularizer=tf.keras.regularizers.l1(0.01),
    activity_regularizer=tf.keras.regularizers.l2(0.01), activation='relu')(gmpl)
batch_norm = BatchNormalization()(dense)
dense_1 = Dense(6, activation='softmax')
output = dense_1(batch_norm)
model = Model(input_, output)


In [ ]:
model.compile(
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  optimizer=tf.keras.optimizers.Adam(lr=0.001),
  metrics=['accuracy']
)
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 50)           1500000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          183296    
_________________________________________________________________
dropout (Dropout)            (None, 100, 256)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 512)          1050624   
_________________________________________________________________
gmpl (GlobalMaxPooling1D)    (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                3283

In [ ]:
#early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

checkpoint = ModelCheckpoint("/content/drive/My Drive/models/amazon_bilstm.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 20,
                          verbose = 1,
                          restore_best_weights = True)

# we put our call backs into a callback list
callbacks = [earlystop, checkpoint]

In [ ]:
padded_data_train.shape

(39414, 100)

In [ ]:
Ytrain.shape

(39414,)

##Model Training

In [ ]:
r = model.fit(padded_data_train,
        Ytrain,
        batch_size=128,
  epochs=10,
  callbacks=callbacks,
  validation_split=VALIDATION_SPLIT
)  

Epoch 1/10
247/247 [==============================] - ETA: 0s - loss: 3.7043 - accuracy: 0.8578
Epoch 00001: val_loss improved from inf to 1.67430, saving model to /content/drive/My Drive/models/amazon_bilstm.h5
247/247 [==============================] - 39s 157ms/step - loss: 3.7043 - accuracy: 0.8578 - val_loss: 1.6743 - val_accuracy: 0.5358
Epoch 2/10
247/247 [==============================] - ETA: 0s - loss: 1.2424 - accuracy: 0.9075
Epoch 00002: val_loss improved from 1.67430 to 1.34637, saving model to /content/drive/My Drive/models/amazon_bilstm.h5
247/247 [==============================] - 37s 149ms/step - loss: 1.2424 - accuracy: 0.9075 - val_loss: 1.3464 - val_accuracy: 0.9045
Epoch 3/10
247/247 [==============================] - ETA: 0s - loss: 1.2110 - accuracy: 0.9298
Epoch 00003: val_loss improved from 1.34637 to 1.20486, saving model to /content/drive/My Drive/models/amazon_bilstm.h5
247/247 [==============================] - 37s 149ms/step - loss: 1.2110 - accuracy: 0.9

##Plotting

In [ ]:
x1=[x for x in range(len(r.history['loss']))]
x2=[x for x in range(len(r.history['loss']))]
x3=[x for x in range(len(r.history['loss']))]
x4=[x for x in range(len(r.history['loss']))]
y1=r.history['loss']
y2=r.history['val_loss']
y3=r.history['accuracy']
y4=r.history['val_accuracy']

In [ ]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))
get_ipython().events.register('pre_run_cell', configure_plotly_browser_state)

##Training and Validation Loss

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
#fig.add_trace(go.Scatter(x=[x for x in range(len(r.history['loss']))], y=r.history['loss'] , mode='lines', name='VOLUME',line=dict(color='#00b8b8', width=2)))
fig.add_trace(go.Scatter(x=x1, y=y1, mode='lines+markers', name='train_loss', line=dict(color='#00b8b8', width=4)))
fig.add_trace(go.Scatter(x=x2, y=y2, mode='lines+markers', name='val_loss', line=dict(color='#e4bd0b', width=4)))
fig.update_layout(width=800,
        height=600,legend=dict(x=.70,y=0.85, traceorder='reversed', font_size=16), 
        yaxis=dict(
        title="Training and Validation Loss",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),)
fig.show()

##Training and Validation Accuracy

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()
#fig.add_trace(go.Scatter(x=[x for x in range(len(r.history['loss']))], y=r.history['loss'] , mode='lines', name='VOLUME',line=dict(color='#00b8b8', width=2)))
fig.add_trace(go.Scatter(x=x3, y=y3, mode='lines+markers', name='train_accuracy', line=dict(color='#00b8b8', width=4)))
fig.add_trace(go.Scatter(x=x4, y=y4, mode='lines+markers', name='val_accuracy', line=dict(color='#e4bd0b', width=4)))
fig.update_layout(width=800,
        height=600,legend=dict(x=.70,y=0.85, traceorder='reversed', font_size=16), 
        yaxis=dict(
        title="Training and Validation Accuracy",
        titlefont=dict(
            color="#1f77b4"
        ),
        tickfont=dict(
            color="#1f77b4"
        )
    ),)
fig.show()

##Saving the Model

In [ ]:
model.save('/gdrive/My Drive/models/amazon_bilstm1.h5')

##Model Predictions

In [ ]:
#from tensorflow.keras.models import load_model
#bilstm = load_model('/content/drive/My Drive/models/amazon_bilstm.h5')
y_hat = model.predict(padded_data_test)
y_hat = np.argmax(y_hat, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
  """
  This function prints and plots the confusion matrix.
  Normalization can be applied by setting `normalize=True`.
  """
  if normalize:
      cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
      print("Normalized confusion matrix")
  else:
      print('Confusion matrix, without normalization')

  print(cm)

  plt.imshow(cm, interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()
  tick_marks = np.arange(len(classes))
  plt.xticks(tick_marks, classes, rotation=45)
  plt.yticks(tick_marks, classes)

  fmt = '.2f' if normalize else 'd'
  thresh = cm.max() / 2.
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
      plt.text(j, i, format(cm[i, j], fmt),
               horizontalalignment="center",
               color="white" if cm[i, j] > thresh else "black")

  plt.tight_layout()
  plt.ylabel('True label')
  plt.xlabel('Predicted label')
  plt.show()

##Model Performance

Confusion matrix, without normalization
[[5154    0   28   19  124   36]
 [  27  457    6   15   70    5]
 [  12    0 1548    2   50   38]
 [  33    3   19 2328  144   18]
 [ 115    4   22   27 7376   49]
 [   4    0   67    9   42 1563]]


In [ ]:
cm = confusion_matrix(Ytest, y_hat)
plot_confusion_matrix(cm, [0, 1, 2,3,4,5])